# The Game of NIM
Nim is a classic mathematical strategy game, variants of which have been played since ancient times. The game itself is rather simple, and you don't need much equiment to play it. The idea is that you have `n` piles of things (seeds, stones, coins, etc.) The first pile has one item in it, the second has two, and so on. You can easily adjust the game by increasing or decreasing the number of piles. For example, a 3 pile game may look like this:

```
1: *
2: **
3: ***
```

In the game of nim, players take turns. On your turn you must remove any number of things from one pile. The object of the game is to not take the last stone. 

## Creating the Board
Our first task is to create a board that can represent and play an `n` pile game of nim. We will also add a handy function to compute all of the possible moves in the game. Moves are specified as the values `r` and `n` which means "take `n` stones from row `r`."

In [1]:
from copy import copy

class Nim:
    def __init__(self, rows=3, board=None):
        """
        Generate a nim board with a given number of rows. If board is specified, 
        it copies the board instead.
        """
        if board is not None:
            self.board = copy(board.board)
        else:
            self.board = []
            for i in range(rows):
                self.board.append(i+1)
    
    def __str__(self):
        """
        Generate a string representation of the nim board
        """
        lines = []
        for i in range(len(self.board)):
            lines.append("{}: {}".format(i+1, '*'*self.board[i]))
        return "\n".join(lines)

    def moves(self):
        """
        Return a list of valid moves as (r,n) tuples where 
        r - Row number
        n - Number of stones to take
        """
        return [(r+1,n) for r in range(len(self.board)) for n in range(1, self.board[r]+1)]
    
    def move(self, r, n):
        """
        Remove n stones from row r.
        """
        r = r-1
        if r>=0 and r<len(self.board) and n <= self.board[r]:
            self.board[r] -= n

## Playing the Game
We will make a general turn based game that can be played between two agents. The game will continue while there are still stones, and it will declare a winner (either player 1 or player 2).

Agent functions will have this form `def agent(board):` The percept for each ply is the current board state. The agent will return a move, which will assume is valid.

In [2]:
def play_nim(n, p1, p2):
    """
    Play the two agent functions against each other in an n row game of nim, 
    printing the board before each move.
    """
    to_move = 1
    last = 2
    board = Nim(n)
    while len(board.moves()) != 0:
        print(board)
        last = to_move
        if to_move == 1:
            r,n = p1(board)
            to_move = 2
        else:
            r,n = p2(board)
            to_move =1
        print("Player {} takes {} from pile {}.".format(last, n, r))
        board.move(r,n)
    print("Player {} wins!".format(to_move))

## A Human Player
Why should we let the machine have all the fun? Let's write a human player for our little nim game! This will prompt the player for an r,n pair and will validate the move. Once we have a valid move, it will return it. 

In [3]:
def human(board):
    done = False
    while not done:
        r,n = eval(input("Enter r,n: "))
        done = (r,n) in board.moves()
    return r,n
    

Let's play a 3 row human-human game of nim.

In [4]:
play_nim(3,human,human)

1: *
2: **
3: ***


Player 1 takes 1 from pile 2.
1: *
2: *
3: ***
Player 2 takes 2 from pile 3.
1: *
2: *
3: *
Player 1 takes 1 from pile 1.
1: 
2: *
3: *
Player 2 takes 1 from pile 2.
1: 
2: 
3: *
Player 1 takes 1 from pile 3.
Player 2 wins!


## The Task: Write a Minimax agent to play nim.
Now, let's create a minimax agent to play the game of nim! Then we'll try it out against human players for a few different sizes.

In [15]:
def minimax(board):
    """
    Return the minimax move for nim.
    """
    def is_terminal(board):
        return sum(board.board) == 1
    
    def max_value(board):
        if is_terminal(board):
            r,n = board.moves()[0]
            return -1,r,n
        vresult = -2
        rresult = 0
        nresult = 0
        for r,n in board.moves():
            # get the result of that move
            trial = Nim(board=board)
            trial.move(r,n)

            v,r1,n1 = min_value(trial)
            if v > vresult:
                vresult = v
                rresult = r
                nresult = n
        return vresult, rresult, nresult
    
    def min_value(board):
        if is_terminal(board):
            r,n = board.moves()[0]
            return 1,r,n
        vresult = 2
        rresult = 0
        nresult = 0
        for r,n in board.moves():
            # get the result of that move
            trial = Nim(board=board)
            trial.move(r,n)

            v,r1,n1 = max_value(trial)
            if v < vresult:
                vresult = v
                rresult = r
                nresult = n
        return vresult, rresult, nresult
    
    # return our maximizing move
    v,r,n = max_value(board)
    return r,n

In [13]:
play_nim(3, human, minimax)

1: *
2: **
3: ***
Player 1 takes 1 from pile 3.
1: *
2: **
3: **
Player 2 takes 1 from pile 1.
1: 
2: **
3: **
Player 1 takes 1 from pile 3.
1: 
2: **
3: *
Player 2 takes 2 from pile 2.
1: 
2: 
3: *
Player 1 takes 1 from pile 3.
Player 2 wins!


In [16]:
play_nim(4, human, minimax)

1: *
2: **
3: ***
4: ****
Player 1 takes 4 from pile 4.
1: *
2: **
3: ***
4: 
Player 2 takes 1 from pile 1.
1: 
2: **
3: ***
4: 
Player 1 takes 1 from pile 3.
1: 
2: **
3: **
4: 
Player 2 takes 1 from pile 2.
1: 
2: *
3: **
4: 
Player 1 takes 2 from pile 3.
1: 
2: *
3: 
4: 
Player 2 takes 1 from pile 2.
Player 1 wins!


In [18]:
play_nim(10, human, minimax)

1: *
2: **
3: ***
4: ****
5: *****
6: ******
7: *******
8: ********
9: *********
10: **********
Player 1 takes 10 from pile 10.
1: *
2: **
3: ***
4: ****
5: *****
6: ******
7: *******
8: ********
9: *********
10: 


KeyboardInterrupt: 